In [1]:
import openai
import pandas as pd

### Embedding Function

In [2]:
def create_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        model=model,
        input=text,
    )
    return response.data[0].embedding


### Read vector embeddings from Local Qdrant

In [12]:
from qdrant_client import QdrantClient


def retrieve_embedding_data(qd_client: QdrantClient, query, collection_name, k=5):
    response = qd_client.query_points(
        collection_name=collection_name,
        query=create_embeddings(query),
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_scores = []
    retrieved_context_ratings = []
    
    for point in response.points:
        retrieved_context_ids.append(point.payload["parent_asin"])
        retrieved_context.append(point.payload["description"])
        retrieved_scores.append(point.score)
        retrieved_context_ratings.append(point.payload["average_rating"])

    # return dictionary of retrieved data
    return {
        "context_ids": retrieved_context_ids,
        "context": retrieved_context,
        "scores": retrieved_scores,
        "context_ratings": retrieved_context_ratings
    }




In [14]:
qdrant_cleint = QdrantClient(
    url="http://localhost:6333",
)

retrieved_context = retrieve_embedding_data(
    qdrant_cleint,
    query="Android 12 Tablet for Kids",
    collection_name="amazon_items-collection-00",
    k=5
)

retrieved_context

{'context_ids': ['B0BCQ8RJG7',
  'B0B5517LRN',
  'B0BL12DTJ5',
  'B0BN7WWH63',
  'B0B2P18Z31'],
 'context': ['Fullant Kids Tablet 7 inch,Android 12 Tablet for Kids,32GB ROM 128GB Expand,Kids Software Pre-Installed,Bluetooth,Dual Camera,Toddler Tablet with Shockproof Case [\'【Newest Android 12 OS 】Fullant tablet for kids is equipped with the latest version of GMS-certified Android 11 GO OS, which is safer and faster than Android 10 and eliminates unwanted ads. The 1.6GHz quad-core processor is 20% faster than regular processors. And supports WiFi, Bluetooth, etc.\', "【Kids\' Safety & Parental Controls】Fullant kids tablets can ensure kids\' safety online with 2 types of parental controls. Using the Kidoz interface, parents can limit screen time, set educational goals, and filter harmful web pages. With the FamilyGroup app, you can check on your kids activity remotely at any time, including how often and how long the tablet was used, to understand your kids usage habits better.", \'【Learn

### Fromat Context embedding + metadata

In [31]:
def format_context(retrived_context):
    formatted_context = ""
    for id, chunk, rating in zip(retrived_context["context_ids"], retrived_context["context"], retrived_context["context_ratings"]):
        formatted_context += f"Product ID: {id}, rating: {rating}, description: {chunk.strip()}\n"
    return formatted_context

In [32]:
new_formatted_context= format_context(retrieved_context)
print(new_formatted_context)

Product ID: B0BCQ8RJG7, rating: 4.0, description: Fullant Kids Tablet 7 inch,Android 12 Tablet for Kids,32GB ROM 128GB Expand,Kids Software Pre-Installed,Bluetooth,Dual Camera,Toddler Tablet with Shockproof Case ['【Newest Android 12 OS 】Fullant tablet for kids is equipped with the latest version of GMS-certified Android 11 GO OS, which is safer and faster than Android 10 and eliminates unwanted ads. The 1.6GHz quad-core processor is 20% faster than regular processors. And supports WiFi, Bluetooth, etc.', "【Kids' Safety & Parental Controls】Fullant kids tablets can ensure kids' safety online with 2 types of parental controls. Using the Kidoz interface, parents can limit screen time, set educational goals, and filter harmful web pages. With the FamilyGroup app, you can check on your kids activity remotely at any time, including how often and how long the tablet was used, to understand your kids usage habits better.", '【Learning, Playing and Reading Included】You can download various YouTub

### Create a promopt

In [ ]:
def build_prompt(preprocessed_context, question):
    prompt = f"""
You are a specialized Product Expert Assistant. Your goal is to answer customer questions accurately using ONLY the provided product information.

### Instructions:
1. **Source of Truth:** Answer strictly based on the provided "Available Products" section below. Do not use outside knowledge or make assumptions.
2. **Handling Missing Info:** If the answer cannot be found in the provided products, politely state that you do not have that information. Do not make up features.
3. **Tone:** Be helpful, professional, and concise.
4. **Terminology:** Never refer to the text below as "context" or "data." Refer to it naturally as "our current inventory" or "available products."

### Available Products:
<inventory_data>
{preprocessed_context}
</inventory_data>

### Customer Question:
{question}

### Answer:
"""
    return prompt

In [34]:
prompt = build_prompt(new_formatted_context, "What kind of tablet is best for kids?")
print(prompt)


You are a specialized Product Expert Assistant. Your goal is to answer customer questions accurately using ONLY the provided product information.

### Instructions:
1. **Source of Truth:** Answer strictly based on the provided "Available Products" section below. Do not use outside knowledge or make assumptions.
2. **Handling Missing Info:** If the answer cannot be found in the provided products, politely state that you do not have that information. Do not make up features.
3. **Tone:** Be helpful, professional, and concise.
4. **Terminology:** Never refer to the text below as "context" or "data." Refer to it naturally as "our current inventory" or "available products."

### Available Products:
<inventory_data>
Product ID: B0BCQ8RJG7, rating: 4.0, description: Fullant Kids Tablet 7 inch,Android 12 Tablet for Kids,32GB ROM 128GB Expand,Kids Software Pre-Installed,Bluetooth,Dual Camera,Toddler Tablet with Shockproof Case ['【Newest Android 12 OS 】Fullant tablet for kids is equipped with t

In [35]:
# Generate a LLM response
import openai

def generate_llm_response(prompt, model="gpt-5-nano"):
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
        ]
    )
    return response.choices[0].message.content




In [36]:
generated_response = generate_llm_response(prompt)
print(generated_response)

For a tablet specifically designed for kids, our best option is the Fullant Kids Tablet 7 inch (B0BCQ8RJG7).

Why it’s great for kids:
- Kid-focused safety: two parental controls (Kidoz and FamilyGroup) to limit screen time and monitor activity.
- Kid-proof design: durable, silicone kid-proof case with built-in kickstand.
- Content and accessibility: Google Play access with learning and entertainment apps pre-installed; 32GB storage expandable up to 128GB.
- Battery and display: 3500mAh battery (up to 8 hours) and a low blue light 1024x600 IPS display with reading mode.
- Warranty and guarantee: 30-day money-back guarantee and 12 months warranty.

If you’re open to a more general-purpose tablet (not kid-specific) that still works well for kids, there’s also the 10-inch Android 12.0 Tablet (B0B5517LRN) with Google Play, 64GB storage (expandable up to 512GB), 8000mAh battery, and dual cameras. It’s not marketed specifically for kids but offers robust features and parental-control options

### Integrated RAG pipeline 

In [38]:
def integrated_rag_pipeline(question, model="gpt-5-nano"):
    
    qdrant_client = QdrantClient(   
        url="http://localhost:6333",
    )
    # Step 1: Retrieve relevant context
    retrieved_context = retrieve_embedding_data(
        qdrant_client,
        question,
        collection_name="amazon_items-collection-00",
        k=5
    )
    # Step 2: Format context
    formatted_context = format_context(retrieved_context)   
    # Step 3: Build prompt
    prompt = build_prompt(formatted_context, question)
    # Step 4: Generate response
    response = generate_llm_response(prompt, model)
    return response

In [40]:
llm_answer = integrated_rag_pipeline("What kind of highly rated user mobile phones?")
print(llm_answer)

We don’t have any mobile phones in our current available products. The closest items are smart watches and a tablet that pair with phones (iOS/Android), with ratings as follows:

- B0BWN4MYDK Smart Watch — rating 4.1
- B0B27Y4J4V Escaru Smart Watch — rating 4.1
- B0B5517LRN Android 12 Tablet — rating 3.5
- B09Q36C75R 1MORE PistonBuds Pro Earbuds — rating 3.8
- B0BW5T1F9J Flash Drive — rating 4.0

If you’d like, I can help identify which of these are compatible with your phone (iOS or Android).
